# sprin13 Kerasその１ 

##  【問題1】公式チュートリアルモデルを分担して実行  
・「ＲＮＮによるテキスト生成」

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [3]:
# 読み込んだのち、Python 2 との互換性のためにデコード
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# テキストの長さは含まれる文字数
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
# テキストの最初の 250文字を参照
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# ファイル中のユニークな文字の数
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


テキストのベクトル化  
訓練をする前に、文字列を数値表現に変換する必要があります。2つの参照テーブルを作成します。  
一つは文字を数字に変換するもの、もう一つは数字を文字に変換するものです。

In [6]:
# それぞれの文字からインデックスへの対応表を作成
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

In [7]:
text_as_int

array([18, 47, 56, ..., 45,  8,  0])

In [15]:
len(text_as_int)

1115394

In [8]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')


{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [9]:
# テキストの最初の 13 文字がどのように整数に変換されるかを見てみる
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


## 最初に tf.data.Dataset.from_tensor_slices 関数を使ってテキストベクトルを文字インデックスの連続に変換します。

In [16]:
# ひとつの入力としたいシーケンスの文字数としての最大の長さ
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# 訓練用サンプルとターゲットを作る
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

###　take は、引数の数だけ取り出すメソッド　###
for i in char_dataset.take(5):
  print(idx2char[i.numpy()])


F
i
r
s
t


array([18, 47, 56, ..., 45,  8,  0])

In [19]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    #repr 最大サイズの制限のある文字列を返す。
    print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


##　最後の一文字を排除したデータ（インプットテキスト）と、最初の１文字を取り除いたデータ（アウトプットテキスト）を出力する

In [20]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

## どうやら ↓ こういうデータで学習させて、次なる文字の予測を重ねて文章生成をしているように見受けられる。

In [26]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [27]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))


Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


##  訓練用バッチの作成  
tf.data を使ってテキストを分割し、扱いやすいシーケンスにします。  
しかし、このデータをモデルに供給する前に、データをシャッフルしてバッチにまとめる必要があります。

In [28]:
# バッチサイズ
BATCH_SIZE = 64

# データセットをシャッフルするためのバッファサイズ
# （TF data は可能性として無限長のシーケンスでも使えるように設計されています。
# このため、シーケンス全体をメモリ内でシャッフルしようとはしません。
# その代わりに、要素をシャッフルするためのバッファを保持しています）
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

モデルの構築  
tf.keras.Sequential を使ってモデルを定義します。この簡単な例では、モデルの定義に3つのレイヤーを使用しています。  
    tf.keras.layers.Embedding: 入力レイヤー。それぞれの文字を表す数を embedding_dim　次元のベクトルに変換する、訓練可能な参照テーブル。  
    tf.keras.layers.GRU: サイズが units=rnn_units のRNNの一種（ここに LSTM レイヤーを使うこともできる）。  
    tf.keras.layers.Dense: vocab_size の出力を持つ、出力レイヤー。
  
  
1文字ごとにモデルは埋め込みベクトルを検索し、その埋め込みベクトルを入力として GRU を 1 タイムステップ実行します。そして Dense レイヤーを適用して、次の文字の対数尤度を予測するロジットを生成します。

In [32]:
# 文字数で表されるボキャブラリーの長さ
vocab_size = len(vocab)
# 埋め込みベクトルの次元
embedding_dim = 256
# RNN ユニットの数
rnn_units = 1024

In [33]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [34]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)


##  モデルを試す

In [35]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [36]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [37]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [38]:
sampled_indices

array([25, 35, 46, 41, 11, 23, 58, 56, 23, 42, 63, 48, 25, 34,  2, 55, 54,
        8, 32, 23, 25,  9, 51, 29, 16, 12, 23, 36,  2, 54, 58, 53, 62, 15,
       40, 21, 38, 10, 32,  0, 43, 10, 61, 17, 55, 28, 52, 53, 13, 53, 20,
       34, 28, 31, 49, 52, 60, 40, 62, 26, 31, 14, 36, 36, 48, 45, 37, 56,
        1, 17, 14, 45,  2, 50, 43, 60, 54, 20, 14, 52, 59, 18, 19, 24, 62,
       22,  7, 39,  8, 62, 24, 51, 10, 14, 44,  3, 16, 57, 64, 49],
      dtype=int64)

In [39]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 "Richard both in shape and mind\nTransform'd and weaken'd? hath Bolingbroke deposed\nThine intellect? h"

Next Char Predictions: 
 'MWhc;KtrKdyjMV!qp.TKM3mQD?KX!ptoxCbIZ:T\ne:wEqPnoAoHVPSknvbxNSBXXjgYr EBg!levpHBnuFGLxJ-a.xLm:Bf$Dszk'


In [40]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1740437


In [41]:
model.compile(optimizer='adam', loss=loss)

In [42]:
# チェックポイントが保存されるディレクトリ
checkpoint_dir = './training_checkpoints'
# チェックポイントファイルの名称
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [43]:
EPOCHS=10

In [44]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 172 steps
Epoch 1/10
172/172 [==============================] - 960s 6s/step - loss: 2.6703
Epoch 2/10
172/172 [==============================] - 926s 5s/step - loss: 1.9677
Epoch 3/10
172/172 [==============================] - 863s 5s/step - loss: 1.6962
Epoch 4/10
172/172 [==============================] - 898s 5s/step - loss: 1.5437
Epoch 5/10
172/172 [==============================] - 858s 5s/step - loss: 1.4560
Epoch 6/10
172/172 [==============================] - 731s 4s/step - loss: 1.3957
Epoch 7/10
172/172 [==============================] - 891s 5s/step - loss: 1.3490
Epoch 8/10
172/172 [==============================] - 816s 5s/step - loss: 1.3107
Epoch 9/10
172/172 [==============================] - 991s 6s/step - loss: 1.2768
Epoch 10/10
172/172 [==============================] - 1389s 8s/step - loss: 1.2427


##　この計算、３時間ぐらいかかった。

最終チェックポイントの復元

予測ステップを単純にするため、バッチサイズ 1 を使用します。

RNN が状態をタイムステップからタイムステップへと渡す仕組みのため、モデルは一度構築されると固定されたバッチサイズしか受け付けられません。

モデルを異なる batch_size で実行するためには、モデルを再構築し、チェックポイントから重みを復元する必要があります。

In [45]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_10'

In [46]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))


In [47]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [85]:
def generate_text(model, start_string):
  # 評価ステップ（学習済みモデルを使ったテキスト生成）

  # 生成する文字数
  num_generate = 200

  # 開始文字列を数値に変換（ベクトル化）
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # 結果を保存する空文字列
  text_generated = []

  # 低い temperature　は、より予測しやすいテキストをもたらし
  # 高い temperature は、より意外なテキストをもたらす
  # 実験により最適な設定を見つけること
  temperature = 1.0

  # ここではバッチサイズ　== 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # バッチの次元を削除
      predictions = tf.squeeze(predictions, 0)

      # カテゴリー分布をつかってモデルから返された文字を予測 
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # 過去の隠れ状態とともに予測された文字をモデルへのつぎの入力として渡す
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))


In [86]:
print(generate_text(model, start_string=u"shit"))

shitter
To my burn.

Nurse:
What couple these title to HENMY BURCH:
More than I can adler bear--

BUCKINGHAM:
Good Pompey, for I have done excuse
Signior bark and purple their strandle citizens
Had in com


In [87]:
print(generate_text(model, start_string=u"shit"))

shits it strays
your honour in his bands; and here be taking of a husband;
But that his bedder what:
Why you can wed being that I'll brink thee an abtion?

COMINIUS:
I thank thee, wench?

First Senator: '


In [88]:
print(generate_text(model, start_string=u"shit"))

shit
In slip on the third!
And that I shall uncurth.

KING RICHARD III:
And two comes the current of high admire
add the envineathe friend and-foldows infecent.
I mean that now I shall lied in parlinal ki


In [ ]:
たわ言
三塁にスリップして!
そして私は呪いを解く。

キング・リチャードIII
そして2つは絶賛の流れ
envineathe friend and foldows infecentを追加します。
私は今傍で嘘をつくつもりだ。